In [71]:
import pandas as pd
from fuzzywuzzy import process
import numpy as np
from IPython.display import display

In [72]:
df1 = pd.read_csv('C://Users//jack//Documents//price_prediction_pipeline//csv//awb_db_complete21.03.2024.csv')
df2 = pd.read_csv('C://Users//jack//Documents//price_prediction_pipeline//dataset//base//base.csv')



# Filter df1 by 'PRODUCT TYPE' == 'LAPTOP'
df1 = df1[df1['ProductType'] == 'LAPTOP']

# Filter df1 to only include Manufacturer and Model
df1 = df1[['Manufacturer', 'Model']]

# Rename columns in df1
df1.columns = ['brand', 'model']

# Lowercase brand and model columns in df1
df1['brand'] = df1['brand'].str.lower()
df1['model'] = df1['model'].str.lower()

# Remove rows where brand ==  apple
df1 = df1[df1['brand'] != 'apple']

# Create new dataframe of unique rows only
df1 = df1.drop_duplicates()

# Strip 'notebook pc' and 'lenovo' from model column in df1
df1['model'] = df1['model'].str.replace('notebook pc', '')
df1['model'] = df1['model'].str.replace('notebook', '')
df1['model'] = df1['model'].str.replace('lenovo', '')
df1['model'] = df1['model'].str.replace('-', '')


# Filter df2 to only include brand and Model
df2 = df2[['brand', 'model']]

# Strip 'notebook pc' and 'lenovo' from model column in df2
df2['brand'] = df2['brand'].str.replace('hp (hewlett-packard)', 'hp')
df2['model'] = df2['model'].str.replace('notebook pc', '')
df2['model'] = df2['model'].str.replace('notebook', '')
df2['model'] = df2['model'].str.replace('lenovo', '')

print(len(df1))
print(df1.head())
# Save df1
df1.to_csv('C://Users//jack//Documents//price_prediction_pipeline//csv//awbc_True_brand_model_pairs.csv', index=False)

print(len(df2))
print(df2.head())
# Save df2
df2.to_csv('C://Users//jack//Documents//price_prediction_pipeline//csv//base_dataset_brand_model_pairs_pre_fuzzy.csv', index=False)

C:\Users\jack\AppData\Local\Temp\ipykernel_1644\3590602451.py:1: DtypeWarning: Columns (33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('C://Users//jack//Documents//price_prediction_pipeline//csv//awb_db_complete21.03.2024.csv')


1259
   brand            model
45  dell  precision m6600
54    hp          255 g7 
63  dell    latitude 5590
64  dell    latitude 5580
66  dell    latitude 7390
8059
  brand            model
0  dell  precision m4800
1  dell  precision m4800
2  dell   precision 3520
3  dell  precision m6600
4  dell   precision 5540


In [73]:
# Create a dictionary from df1 where models are keys and brands are values
model_brand_dict = df1.set_index('model')['brand'].to_dict()

# drop rows with missing model values
df2 = df2.dropna(subset=['model'])

# drop rows with empty string values in model column
df2 = df2[df2['model'] != '']

def get_corresponding_brand(row):
    model = row['model']
    if isinstance(model, str):
        match = process.extractOne(model, df1['model'].unique(), scorer=process.fuzz.token_sort_ratio, score_cutoff=90)
        return model_brand_dict.get(match[0]) if match else 'No Match'
    else:
        return 'No Match'

# Apply the function to the DataFrame df2
df2['brand'] = df2.apply(get_corresponding_brand, axis=1)

# save df2
df2.to_csv('C://Users//jack//Documents//price_prediction_pipeline//csv//base_dataset_brand_model_pairs_post_fuzzy.csv', index=False)


In [74]:
display(df2)

# Now compare df1 with df2 and create new df with differences
df3 = pd.concat([df1, df2]).drop_duplicates(keep=False)
display(df3)


,brand,model
0,dell,precision m4800
1,dell,precision m4800
2,dell,precision 3520
3,dell,precision m6600
4,dell,precision 5540
...,...,...
8054,hp,probook 450 g7
8055,hp,probook 430 g3
8056,dell,inspiron 15 5510
8057,dell,inspiron 15 5510


,brand,model
855,lenovo,v11015isk
865,lenovo,ideapad 10015iby
868,lenovo,invalid
1963,lenovo,v13014ikb
2461,acer,travelmate p21553
...,...,...
7820,No Match,satellite pro c50-a-1mx
7927,No Match,latitude 5310 2-in-1
8009,No Match,yoga 510-14isk
8010,No Match,yoga 710-14ikb
